# Deploying a Large Language Model in the GenAI Hub of SAP AI Core on BTP

## Pre-requisites

- Have [python](https://www.python.org/downloads/) installed
- Create an instance of SAP AI Core in your BTP sub-account: [documentation](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/initial-setup?locale=en-US)
- Find information about available LLM's in [SAP Note 3437766](https://me.sap.com/notes/3437766) 
- Optionally, have the [Cloud Foundry CLI installed](https://docs.cloudfoundry.org/cf-cli/install-go-cli.html)


In [ ]:
# Install the AI Core SDK with pip

%pip install ai_core_sdk

In [ ]:
# In your working directory, log on to BTP with cf logon
# Download the credentials of your AICore instance with cf service-key <service-name> <key-name> > key.json
# Remove the first line of the key.json file
# Alternatively, create and download a service key in the BTP cockpit 

import json

with open('key.json') as f:
    btp_key = json.load(f).get('credentials')   # when using cf service-key
    # btp_key = json.load(f)                    # when using a downloaded service key from BTP cockpit

print(btp_key["serviceurls"]["AI_API_URL"])

In [42]:
# Load AI Core SDK
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection using credentials from downloaded key.json
ai_core_client = AICoreV2Client(
    base_url = btp_key["serviceurls"]["AI_API_URL"] + "/v2", # The present SAP AI Core API version is 2
    auth_url=  btp_key["url"] + "/oauth/token", # Suffix to add
    client_id = btp_key["clientid"],
    client_secret = btp_key["clientsecret"],
)


In [ ]:
# Query existing resource groups. It is expected that group "default" is present
response = ai_core_client.resource_groups.query()

for rg in response.resources:
    print(rg.resource_group_id)

In [ ]:
# Find available models and their corresponding exectuables
# Alternatively, find this information in SAP Note 3437766 - https://me.sap.com/notes/3437766

exc = ai_core_client.executable.query(resource_group="default", scenario_id="foundation-models")

for e in exc.resources:
    p = e.parameters
    for i in p: 
        print (f"Executable: {e.id} - {i.description}")



In [ ]:
# Create a new Configuration for OpenAI gpt-4o

from ai_api_client_sdk.models.parameter_binding import ParameterBinding

model_to_deploy = "gpt-4o"
executable_id = "azure-openai"

pb1 = ParameterBinding(key="modelName", value=model_to_deploy)
pb2 = ParameterBinding(key="modelVersion", value="latest")

ai_core_client.configuration.create( name=model_to_deploy, executable_id=executable_id, scenario_id="foundation-models",resource_group="default", parameter_bindings=[pb1, pb2])

In [ ]:
# Query existing Configurations
confs = ai_core_client.configuration.query(scenario_id="foundation-models", resource_group="default")

for resource in confs.resources:
    print(f"Configuration ID: {resource.id}, Executable ID: {resource.executable_id}, Name: {resource.name}, Param0: {resource.parameter_bindings[0].value}, Param1: {resource.parameter_bindings[1].value}")


In [46]:
# Attention: make sure to use the correct configuration id which corresponds to the model you want to deploy

conf_id = "4be0f740-3d39-4522-aa90-0cb2061c4e1d"

# Create a deployment for the configuration
ai_core_client.deployment.create(configuration_id=conf_id, resource_group="default")

In [ ]:
# Optional: Find existing deployments for a configuration

deps = ai_core_client.deployment.query(resource_group="default", configuration_id=conf_id)

print(deps)

In [ ]:
# Optional: Get details of a deployment

for dep in deps.resources:
    print(dep.id)
    dep_detail = ai_core_client.deployment.get(deployment_id=dep.id, resource_group="default")
    print(dep_detail.details['resources']['backend_details']['model']['name'])
    print(dep_detail.details['resources']['backend_details']['model']['version'])
